# Unitary ansatz entering the VQE

The final energy output of a VQE calculation will crucially depend on the ansatz/form of the parameterized unitary $\hat U(\boldsymbol{\theta})$ employed in state preparation. Here we review two popular approaches, the unitary coupled cluster and qubit coupled cluster methodologies, and benchmark them for energy calculations of small molecules.

In [1]:
import numpy as np
import tequila as tq
from utility import *
threshold = 1e-6 #Cutoff for UCC MP2 amplitudes and QCC ranking gradients

## Unitary Coupled Cluster (UCC)

The UCC ansatz is obtained by 'unitarizing' the traditional coupled cluster ansatz,
$$ e^{\hat T} \rightarrow e^{\hat T - \hat T^\dagger} \equiv \hat U_{\text{UCC}}$$


Due to non-commutativity of terms in $\hat T - \hat T^\dagger$, the UCC ansatz does not have a straightforward decomposition in terms of circuit primitives implementable on the quantum computer. Therefore, to obtain a form which can be compiled, we employ the Trotter approximation. The accuracy of the circuit ansatz relative to the exact UCC operator will be dependent on how many Trotter steps are employed. The number of Trotter steps is commonly set to its minimal value of one to avoid excessive circuit depth.

In [2]:
trotter_steps = 1

### H2 in STO-3G basis

Below is a sample VQE simulation using the UCCSD ansatz compiled using a single trotter step for H$_2$ in minimal basis at $R=2.5$ (Angstrom). For comparison, we can run FCI to obtain the true ground state energy.


In [3]:
xyz_data = get_molecular_data('h2', geometry=2.5, xyz_format=True)
basis='sto-3g'

h2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

print('Number of spin-orbitals (qubits): {} \n'.format(2*h2.n_orbitals))

E_FCI = h2.compute_energy(method='fci')

print('FCI energy: {}'.format(E_FCI))

Number of spin-orbitals (qubits): 4 

FCI energy: -0.9360549199436616


The UCCSD VQE optimization is of the form
$$E = \min_{\boldsymbol{\theta}} \langle \text{HF} | \hat U_{\text{UCC}}^\dagger(\boldsymbol{\theta}) \hat H  \hat U_{\text{UCC}} (\boldsymbol{\theta}) | \text{HF} \rangle $$
To expedite the optimization process, we can set the initial guess of the amplitudes to zero, i.e. the optimization will begin at the Hartree Fock state. This heuristic is best suited for when Hartree Fock is believed to be in qualitative agreement with the true ground state. To further alleviate quantum resources, we can estimate the amplitudes using classical electronic structure methods (here, MP2 perturbation theory), and only include the unitaries with non-zero estimated amplitudes.

In [4]:
H = h2.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = h2.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-6)

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 15 terms


Number of UCCSD amplitudes: 1 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 32 expectationvalues

active variables : 1

E=-0.70294360  angles= {(1, 0, 1, 0): 0.0}  samples= None
E=-0.92698008  angles= {(1, 0, 1, 0): -0.5644201636314392}  samples= None
E=-0.93354955  angles= {(1, 0, 1, 0): -0.756477668453395}  samples= None
E=-0.93605486  angles= {(1, 0, 1, 0): -0.6900732739717247}  samples= None
E=-0.93605492  angles= {(1, 0, 1, 0): -0.6904080632337539}  samples= None
E=-0.93605492  angles= {(1, 0, 1, 0): -0.6904070264134937}  samples= None
Optimization terminated successfully.
         Current function value: -0.936055
         Iterations: 5
         Function evaluations: 6
         Gradient evaluations: 6

Obtained UCCSD energy: -0.9

We see that the converged UCCSD energy is in exact agreement with the FCI energy, as expected for a $2$-electron system. 

### H2O in 6-31G basis

Now let us try a larger problem of H$_2$O in 6-31G basis. However, we will restrict the active space. The unrestricted problem leads to a $14$-qubit Hamiltonian, and $34$ UCCSD amplitudes to optimize even after neglecting the zero MP2 amplitudes. Therefore, we will remove some orbital degrees of freedom which are less important in accurately describing the electronic structure. By freezing all orbitals other than $0b_1$, $1b_1$, $2a_1$, and $3a_1$, we reduce the problem to an $8$-qubit Hamiltonian with $8$ UCCSD variational amplitudes.

In [5]:
xyz_data = get_molecular_data('h2o', geometry=1, xyz_format=True)

basis = '6-31g'
active = {'B1':[0,1], 'A1':[2,3]}
h2o = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, active_orbitals = active)

print('Number of spin-orbitals (qubits): {} \n'.format(2*h2o.n_orbitals))

E_FCI = h2o.compute_energy(method='fci')

print('FCI energy: {}'.format(E_FCI))

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
Number of spin-orbitals (qubits): 8 

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for fci.
FCI energy: -75.98980140961186


We will then run the UCCSD VQE simulation (***warning: tq.minimize will take several minutes - 1 hour + to converge for a VQE instance of this size.*** Smaller active spaces can be employed to lower VQE simulation runtimes).

In [6]:
H = h2o.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = h2o.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-4)

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 185 terms


Number of UCCSD amplitudes: 8 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 512 expectationvalues

active variables : 8

E=-75.98071144  angles= {(3, 0, 3, 0): 0.0, (3, 1, 2, 0): 0.0, (2, 1, 2, 1): 0.0, (2, 0, 3, 1): 0.0, (3, 1, 3, 1): 0.0, (3, 0, 2, 1): 0.0, (2, 0, 2, 0): 0.0, (2, 1, 3, 0): 0.0}  samples= None
E=-75.57991345  angles= {(3, 0, 3, 0): -0.1516876220703125, (3, 1, 2, 0): -0.04468536376953125, (2, 1, 2, 1): -0.06198883056640625, (2, 0, 3, 1): -0.04468536376953125, (3, 1, 3, 1): -0.0497894287109375, (3, 0, 2, 1): 0.15348052978515625, (2, 0, 2, 0): -0.06015777587890625, (2, 1, 3, 0): 0.15348052978515625}  samples= None
E=-75.98420833  angles= {(3, 0, 3, 0): -0.012046029637663564, (3, 1, 2, 0): -0.0035486166174326273, (2, 1, 

We obtain a UCCSD energy within chemical accuracy (< 1.6 miliHartree from FCI energy) for the H$_2$O molecule near equilibrium geometry. As the bond distance approaches the dissociation limit, the energy deviation from FCI is typically expected to increase as electronic correlations increase during covalent bond-breaking. Furthermore, as HF becomes energetically more distant from the FCI wavefunction, the initial guess of all amplitudes being zero may lead to a local minimum. One can then instead initialize the amplitudes using random guesses, and repeat for $n$ samples to attempt to find the global minimum. To facilitate this, one can run the commented lines below. This will execute $n$ instances of tq.minimize for uniformly random initial parameters in interval $[0, 2\pi]$ and return the lowest optimization result.

In [ ]:
#n=10
#result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

## Qubit Coupled Cluster (QCC)

In contrast to UCC, the QCC methodology makes no direct reference to fermionic algebra and seeks to construct an efficient ansatz directly in qubit-space by finding multi-qubit Pauli strings (entanglers) which lower energy. This is done through an energy-lowering heuristic employing the energy gradient with respect to a Pauli strings variational amplitude. As opposed to UCCSD, the circuit depth and number of parameter is chosen to meet hardware limitations, i.e. one must choose how many exponentiated Pauli strings will be entering the QCC ansatz.

### H2 in STO-3G basis

Below we perform the entangler screening protocol for H2 in minimal basis, and obtain one grouping of entanglers with non-zero energy gradient. We then select one of them to be used in the QCC VQE simulation.

In [16]:
xyz_data = get_molecular_data('h2', geometry=2.5, xyz_format=True)
basis='sto-3g'

h2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set='sto-3g')

hf_reference = hf_occ(2*h2.n_orbitals, h2.n_electrons)

H = h2.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 1

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*h2.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*h2.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)




Hamiltonian has 15 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.2822

Selected entanglers:
1.0 [X0 Y1 X2 X3]


Once the QCC ranking procedure has been ran, we can simulate the QCC VQE optimization with the generated entanglers. The VQE optimization for the QCC ansatz is of the form
$$E = \min_{\boldsymbol{\Omega}, \boldsymbol{\tau}} \langle \boldsymbol{\Omega} | U_{\text{ENT}}^\dagger (\boldsymbol{\tau}) \hat H  U_{\text{ENT}} (\boldsymbol{\tau}) | \boldsymbol{\Omega} \rangle $$
where $\boldsymbol{\Omega}$ denote collective Euler angles parameterizing single-qubit rotations, and $\boldsymbol{\tau}$ are entangler amplitudes. 

In [17]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*h2.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-6)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 18 expectationvalues

active variables : 9

E=-0.70294360  angles= {gamma_3: 0.0, beta_1: 3.141592653589793, gamma_0: 0.0, tau_0: 0.0, gamma_2: 0.0, gamma_1: 0.0, beta_2: 0.0, beta_0: 3.141592653589793, beta_3: 0.0}  samples= None
E=-0.77938639  angles= {gamma_3: 0.0, beta_1: 3.141592653589793, gamma_0: 0.0, tau_0: 0.2822100520133972, gamma_2: 0.0, gamma_1: 0.0, beta_2: 0.0, beta_0: 3.141592653589793, beta_3: 0.0}  samples= None
E=-0.93592356  angles= {gamma_3: 0.0, beta_1: 3.141592653589793, gamma_0: 0.0, tau_0: 1.411050260066986, gamma_2: 0.0, gamma_1: 0.0, beta_2: 0.0, beta_0: 3.141592653589793, beta_3: 0.0}  samples= None
E=-0.93603455  angles= {gamma_3: 0.0, beta_1: 3.141592653589793, gamma_0: 0.0, tau_0: 1.3689076316049542, gamma_2: 0.0, 

We see that the QCC energy converged to the FCI energy with only a single entangler! 

### H2O in 6-31G basis

Let us move on to the problem of H$_2$O in 6-31G basis using the same active space as the UCCSD example.

In [6]:
xyz_data = get_molecular_data('h2o', geometry=1, xyz_format=True)

basis = '6-31g'
active = {'B1':[0,1], 'A1':[2,3]}
h2o = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, active_orbitals = active)
hf_reference = hf_occ(2*h2o.n_orbitals, h2o.n_electrons)


H = h2o.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 6

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*h2o.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))

entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*h2o.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.

Hamiltonian has 185 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.0758
2 : 0.0371
3 : 0.0371
4 : 0.031
5 : 0.0301
6 : 0.0249
7 : 0.0198
8 : 0.0198
9 : 0.0173
10 : 0.0173
11 : 0.0
12 : 0.0

Selected entanglers:
1.0 [X0 Y1 X6 X7]
1.0 [X0 Y3 X5 X6]
1.0 [X1 Y2 X4 X7]
1.0 [X2 Y3 X4 X5]
1.0 [X0 Y1 X4 X5]
1.0 [X2 Y3 X6 X7]


With the 6 chosen entanglers, we can then run the QCC VQE simulation.

In [7]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*h2o.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4)


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 44 expectationvalues

active variables : 22

E=-75.98071144  angles= {gamma_7: 0.0, tau_0: 0.0, gamma_5: 0.0, beta_1: 3.141592653589793, tau_3: 0.0, tau_5: 0.0, tau_4: 0.0, gamma_1: 0.0, beta_0: 3.141592653589793, gamma_3: 0.0, gamma_4: 0.0, beta_5: 0.0, beta_2: 3.141592653589793, beta_4: 0.0, gamma_2: 0.0, beta_7: 0.0, tau_1: 0.0, tau_2: 0.0, gamma_6: 0.0, beta_3: 3.141592653589793, beta_6: 0.0, gamma_0: 0.0}  samples= None
E=-75.98783364  angles= {gamma_7: 0.0, tau_0: 0.07584381103515625, gamma_5: 0.0, beta_1: 3.141592653589793, tau_3: 0.030994415283203125, tau_5: 0.02489471435546875, tau_4: 0.030078887939453125, gamma_1: 0.0, beta_0: 3.141592653589793, gamma_3: 0.0, gamma_4: 0.0, beta_5: 0.0, beta_2: 3.141592653589793, beta_4: 0.0, gamma_2: 

We obtain chemical accuracy for water near equilibrium geometry with only 6 entanglers. The obtained energy is not as accurate as that of UCCSD for this problem, however the QCC optimization may be performed at a fraction of the UCCSD circuit depth. One can also increase the number of entanglers entering the QCC ansatz to increase accuracy. As a final check, one can always run $n$ VQE trials with random initial guesses to test if the optimization fell into a local minimum. ***(Warning: Completing n=10 trials may take a few minutes for this VQE instance).***

In [8]:
n = 10
result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 44 expectationvalues

active variables : 22

E=-74.72863483  angles= {gamma_7: 1.2943697738877236, tau_0: 4.988480926540945, gamma_5: 0.9187555063348196, beta_1: 1.829015364772612, tau_3: 5.66076752282859, tau_5: 6.188245020859796, tau_4: 3.271842760348092, gamma_1: 5.386789573793364, beta_0: 2.16863319397154, gamma_3: 6.229607259068195, gamma_4: 3.8082917717557754, beta_5: 5.928799869627327, beta_2: 1.5977383419153086, beta_4: 1.4267087149043718, gamma_2: 1.5785345140689353, beta_7: 4.925899849934009, tau_1: 3.037487940287173, tau_2: 0.4091435985766853, gamma_6: 5.485195124612168, beta_3: 5.740555378623027, beta_6: 5.966738647629863, gamma_0: 3.9960084393454562}  samples= None
E=-74.85002750  angles= {gamma_7: 1.2934428024521767, tau_0: 5.0634

E=-75.62839475  angles= {gamma_7: 1.2706649773965155, tau_0: 6.339000752507472, gamma_5: 0.9630715044084109, beta_1: 0.5646461938777421, tau_3: 6.53281119912796, tau_5: 6.283535230067367, tau_4: 3.2081049370088386, gamma_1: 5.737025920754317, beta_0: 2.9250439636054133, gamma_3: 6.151060161168804, gamma_4: 3.938118614158134, beta_5: 6.300468446288158, beta_2: 2.730488287421144, beta_4: 3.354364116671956, gamma_2: 1.2142174918422515, beta_7: 6.12353560044104, tau_1: 3.218356743687886, tau_2: 0.0980078096585564, gamma_6: 5.536399615040599, beta_3: 5.808188078734233, beta_6: 5.951359627319547, gamma_0: 3.5928539970202986}  samples= None
E=-75.69018394  angles= {gamma_7: 1.2735452461021715, tau_0: 6.299188914012804, gamma_5: 0.9658460978336706, beta_1: 0.3451529303487214, tau_3: 6.331495378926041, tau_5: 6.258185437348438, tau_4: 3.1577911432275965, gamma_1: 5.765294647044952, beta_0: 3.076236909382921, gamma_3: 6.145697271160691, gamma_4: 3.942231756951102, beta_5: 6.294562402040305, beta

E=-75.98840776  angles= {gamma_7: 1.2551720921122225, tau_0: 6.379343312695238, gamma_5: 0.9813536045804312, beta_1: -0.001083244143025533, tau_3: 6.335262036943046, tau_5: 6.320386778154016, tau_4: 3.1832467798591795, gamma_1: 5.841414989427989, beta_0: 3.158226832932796, gamma_3: 6.125787764448888, gamma_4: 3.942067214079448, beta_5: 6.268280219596934, beta_2: 3.151835598975628, beta_4: 3.111732188563827, gamma_2: 1.1501652183839461, beta_7: 6.285850396047465, tau_1: 3.2016823672998695, tau_2: -0.061337771270136794, gamma_6: 5.568732402369931, beta_3: 6.264567825185274, beta_6: 3.140980178753397, gamma_0: 3.5116036743180934}  samples= None
E=-75.98851162  angles= {gamma_7: 1.2550930295032898, tau_0: 6.379293560745065, gamma_5: 0.9813025809725847, beta_1: -0.004503947813456078, tau_3: 6.335678958035332, tau_5: 6.31878460213419, tau_4: 3.1883138513220772, gamma_1: 5.840824577708186, beta_0: 3.1475800890324095, gamma_3: 6.125866189265665, gamma_4: 3.9418446938254452, beta_5: 6.272229400

E=-75.35996139  angles= {gamma_7: 2.673602723391332, tau_0: 0.20669531439515648, gamma_5: 6.171814153766129, beta_1: 3.448706634662842, tau_3: 0.8971758395796807, tau_5: 1.5352571551904468, tau_4: 0.00016865058794097304, gamma_1: 2.362299456053562, beta_0: -0.09486937111519252, gamma_3: 1.0537331777179262, gamma_4: 6.034046507045286, beta_5: 1.0293439325280627, beta_2: 0.08161200860924026, beta_4: 0.5461999678391285, gamma_2: 4.347844862229412, beta_7: 3.2574736262618926, tau_1: 3.1275668257742897, tau_2: 0.03911310244528238, gamma_6: 4.519863046374728, beta_3: 5.996004488728047, beta_6: 3.5081654507794706, gamma_0: 2.3967609518278103}  samples= None
E=-75.40685292  angles= {gamma_7: 2.6899879973113885, tau_0: 0.08367485020247767, gamma_5: 6.153808060188023, beta_1: 3.1651763288222745, tau_3: 0.8773950638476917, tau_5: 1.538353513651257, tau_4: -0.024644411108648633, gamma_1: 2.360610426062775, beta_0: -0.15104344427622302, gamma_3: 1.0729548081777003, gamma_4: 6.015037524701769, beta_

E=-75.49484880  angles= {gamma_7: 2.6862717904904563, tau_0: -0.008922866850641304, gamma_5: 6.159886191397479, beta_1: 3.109921121451753, tau_3: -0.050693944588363155, tau_5: -0.12115743377093938, tau_4: 0.02325564280612216, gamma_1: 2.359072543378862, beta_0: 0.036314767899986444, gamma_3: 1.0649260806760255, gamma_4: 6.0202509862809945, beta_5: 1.8012266042693084, beta_2: 0.05399819361849944, beta_4: 0.10978605207195333, gamma_2: 4.347211792085754, beta_7: 2.8203187604880364, tau_1: 3.118488627571474, tau_2: -0.22331910147736314, gamma_6: 4.525477684951447, beta_3: 6.313577861162747, beta_6: 3.215640996197584, gamma_0: 2.390220520485301}  samples= None
E=-75.50037658  angles= {gamma_7: 2.6831627614607525, tau_0: -0.0008678365356464442, gamma_5: 6.163694678228094, beta_1: 3.1404099440902247, tau_3: -0.05805789733057469, tau_5: -0.26754234485339373, tau_4: 0.014924700624767992, gamma_1: 2.3592198598478293, beta_0: 0.09400690574751054, gamma_3: 1.060952509978132, gamma_4: 6.02420614591

...

E=-75.98720501  angles= {gamma_7: 4.894845652476635, tau_0: 3.0436864504101453, gamma_5: 2.150894483553162, beta_1: 3.1356604630750655, tau_3: 6.346953945323925, tau_5: 0.03433546530061184, tau_4: 0.060470341177767756, gamma_1: 5.903767756012241, beta_0: 0.003525377888578665, gamma_3: 5.986983898552143, gamma_4: 3.4292975741266467, beta_5: 6.3186155359996015, beta_2: 6.261933378638672, beta_4: 3.1320005407240585, gamma_2: 3.7805950360977882, beta_7: 0.013565268190263928, tau_1: 6.348279466787839, tau_2: 3.1873703918033023, gamma_6: 0.8687646618433119, beta_3: 3.224006638015011, beta_6: 3.130654950006215, gamma_0: 5.611139903594408}  samples= None
E=-75.98802005  angles= {gamma_7: 4.895034407768232, tau_0: 3.042616378337805, gamma_5: 2.150658971376286, beta_1: 3.1381639294856187, tau_3: 6.345736551117281, tau_5: 0.03441615167175394, tau_4: 0.048192560818309436, gamma_1: 5.903793078047397, beta_0: 0.0019478336023704382, gamma_3: 5.986495149360479, gamma_4: 3.431834069978608, beta_5: 6.31

E=-75.15414850  angles= {gamma_7: 2.0601190839365673, tau_0: -0.0933518503598824, gamma_5: 5.454830641683024, beta_1: 3.9126413440596353, tau_3: 1.4330837243126882, tau_5: 3.6843924366325016, tau_4: 3.1130624790583736, gamma_1: 4.950179213409187, beta_0: 2.541766512841683, gamma_3: 4.16416011941808, gamma_4: 2.1497786988068843, beta_5: 5.298452138963759, beta_2: 0.5924508917525408, beta_4: 3.901012344647448, gamma_2: 4.190638866157874, beta_7: 5.352526444943687, tau_1: 2.7054822406576196, tau_2: 3.4822456206644503, gamma_6: 3.228600799987521, beta_3: 2.929090674049153, beta_6: 5.612344401346765, gamma_0: 3.8500723043048732}  samples= None
E=-75.41931315  angles= {gamma_7: 2.084825297220438, tau_0: 0.041152391585183926, gamma_5: 5.522125640407094, beta_1: 3.422018380866351, tau_3: 1.3084329969910333, tau_5: 3.541758002836603, tau_4: 3.110954993690113, gamma_1: 4.929495200139903, beta_0: 3.045922491010354, gamma_3: 4.178283029044692, gamma_4: 2.167781444438961, beta_5: 5.434507360275813,

E=-75.58594859  angles= {gamma_7: 2.031590231305264, tau_0: -0.09940572031338636, gamma_5: 5.537505856894502, beta_1: 3.0502694037003413, tau_3: -0.24990751563244684, tau_5: 3.1842731389489796, tau_4: 3.145587986777994, gamma_1: 4.893378802918953, beta_0: 3.173634558588894, gamma_3: 4.2252374624964855, gamma_4: 2.03135496902317, beta_5: 6.635966207516024, beta_2: -0.5040074167562554, beta_4: 2.7580181940742494, gamma_2: 4.0899428931777395, beta_7: 6.201128624699781, tau_1: 3.0552249029051355, tau_2: 2.8631849258026416, gamma_6: 3.0554442180813344, beta_3: 3.067423941821875, beta_6: 6.284518575393961, gamma_0: 3.8250535429641674}  samples= None
E=-75.59954395  angles= {gamma_7: 2.031266269821733, tau_0: -0.08181539576159055, gamma_5: 5.534751473931125, beta_1: 3.1493870827537034, tau_3: -0.33778267735252204, tau_5: 3.1031511853687164, tau_4: 3.15138462565771, gamma_1: 4.892963213728822, beta_0: 3.1189583180755895, gamma_3: 4.22665019760347, gamma_4: 2.037067285700308, beta_5: 6.50607752

E=-75.88707368  angles= {gamma_7: 2.035599976547116, tau_0: -0.028773564958509808, gamma_5: 5.5657993805566175, beta_1: 3.2998604720371354, tau_3: 0.14196379126995887, tau_5: 3.0754508739183137, tau_4: 3.185070546972301, gamma_1: 4.877172761628566, beta_0: 2.7962475048133624, gamma_3: 4.227708712799843, gamma_4: 2.157897794988302, beta_5: 6.377368388056039, beta_2: -3.4267331858130525, beta_4: 5.27642313160007, gamma_2: 3.9829024532406705, beta_7: 6.268398469889768, tau_1: 3.267905161940767, tau_2: 3.376898163204478, gamma_6: 3.029402202657421, beta_3: 3.090111696068022, beta_6: 6.2680682710657365, gamma_0: 3.811125156332432}  samples= None
E=-75.90920342  angles= {gamma_7: 2.0426041897623017, tau_0: -0.015793614154601593, gamma_5: 5.578413251687956, beta_1: 3.3068570136471367, tau_3: 0.11012059086795198, tau_5: 3.062972198832205, tau_4: 3.203013409100112, gamma_1: 4.87627186816564, beta_0: 2.778731515349791, gamma_3: 4.225577915942652, gamma_4: 2.1647400101959167, beta_5: 6.4924567716

E=-75.98858338  angles= {gamma_7: 2.0812586412269583, tau_0: -0.09849169561647544, gamma_5: 5.638337250108185, beta_1: 3.1414777791855863, tau_3: 0.048163058230757186, tau_5: 3.1758870090829614, tau_4: 3.1830356957809274, gamma_1: 4.873100598674304, beta_0: 3.142022638378164, gamma_3: 4.220197892357346, gamma_4: 2.2346518290144424, beta_5: 6.283173393817604, beta_2: -3.141612904477417, beta_4: 6.2832349787647725, gamma_2: 3.948417588116114, beta_7: 6.284080441517276, tau_1: 3.202703880297173, tau_2: 3.204548059088131, gamma_6: 2.989210589562406, beta_3: 3.1407894089407797, beta_6: 6.282246895881384, gamma_0: 3.7698191594473816}  samples= None
E=-75.98858361  angles= {gamma_7: 2.0812637611941764, tau_0: -0.09842564939187269, gamma_5: 5.638351664898446, beta_1: 3.1417172742304804, tau_3: 0.048115243750170025, tau_5: 3.175908807772808, tau_4: 3.1829422276241957, gamma_1: 4.873098407075037, beta_0: 3.14191609519839, gamma_3: 4.220199551513648, gamma_4: 2.234659401209805, beta_5: 6.28351498